In [9]:
# Geocodificação e Enriquecimento de Endereços - VERSÃO 2.2
# SEM BOUNDING BOX - foco em queries mais inteligentes

# %% [markdown]
# # 1. Configuração Inicial

# %%
import pandas as pd
import json
import time
import re
from pathlib import Path
from datetime import datetime
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderServiceError
import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

print("=" * 80)
print("GEOCODIFICAÇÃO E ENRIQUECIMENTO DE ENDEREÇOS - v2.2")
print("=" * 80)
print(f"\n✓ Bibliotecas importadas")
print(f"✓ Iniciado em: {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}")

# %% [markdown]
# # 2. Configurar Estrutura

# %%
print("\n" + "=" * 80)
print("CONFIGURANDO AMBIENTE")
print("=" * 80)

# Definir diretório base
script_dir = Path(__file__).parent if '__file__' in locals() else Path.cwd()
project_root = script_dir.parent if script_dir.name == 'notebooks' else script_dir

# Criar pastas
pasta_geocoded = project_root / 'data' / 'geocoded'
pasta_geocoded.mkdir(exist_ok=True, parents=True)

pasta_raw = project_root / 'data' / 'raw'

# Arquivos
cache_file = pasta_geocoded / 'cache_geocodificacao.json'

# Configurar geocodificador
geolocator = Nominatim(
    user_agent="sp_centro_geocoder_v2",
    timeout=15
)

print(f"\n✓ Estrutura de pastas:")
print(f"  - Raiz: {project_root}")
print(f"  - Raw: {pasta_raw}")
print(f"  - Geocoded: {pasta_geocoded}")
print(f"✓ Geocodificador: Nominatim (OpenStreetMap)")
print(f"⚠️  Limite: 1 requisição/segundo")

# %% [markdown]
# # 3. Carregar Planilha

# %%
print("\n" + "=" * 80)
print("CARREGANDO PLANILHA")
print("=" * 80)

# Verificar se a pasta raw existe
if not pasta_raw.exists():
    print(f"❌ Pasta não encontrada: {pasta_raw}")
    print("📁 Estrutura atual do projeto:")
    for item in project_root.rglob('*'):
        if item.is_dir():
            print(f"  📂 {item.relative_to(project_root)}")
    raise Exception(f"Pasta 'data/raw' não encontrada em {project_root}")

arquivos = list(pasta_raw.glob('*.xlsx'))

if arquivos:
    # Ordenar por data de modificação
    arquivos = sorted(arquivos, key=lambda x: x.stat().st_mtime, reverse=True)
    
    print(f"\n📁 Arquivos disponíveis em {pasta_raw}:\n")
    for i, arq in enumerate(arquivos, 1):
        modificado = datetime.fromtimestamp(arq.stat().st_mtime).strftime('%d/%m/%Y %H:%M')
        tamanho = arq.stat().st_size / 1024 / 1024  # MB
        print(f"  {i}. {arq.name}")
        print(f"     ↳ Modificado: {modificado} | Tamanho: {tamanho:.1f} MB")
    
    print("\n" + "=" * 80)
    
    # Se só tem um arquivo, seleciona automaticamente
    if len(arquivos) == 1:
        arquivo = arquivos[0]
        print(f"\n✓ Único arquivo encontrado: {arquivo.name}")
    else:
        escolha = input("Digite o NÚMERO do arquivo: ")
        try:
            idx = int(escolha) - 1
            arquivo = arquivos[idx]
            print(f"\n✓ Selecionado: {arquivo.name}")
        except:
            raise Exception("Seleção inválida!")
else:
    print(f"❌ Nenhum arquivo .xlsx encontrado em: {pasta_raw}")
    print("📁 Conteúdo da pasta raw:")
    if pasta_raw.exists():
        for item in pasta_raw.iterdir():
            tipo = "📂" if item.is_dir() else "📄"
            print(f"  {tipo} {item.name}")
    raise Exception(f"Nenhum arquivo Excel encontrado em {pasta_raw}")

# Carregar dados
print(f"\n📊 Carregando arquivo...")
df = pd.read_excel(arquivo)
print(f"✓ {len(df):,} registros carregados de {arquivo.name}")

# Identificar coluna de logradouro
col_logradouro = None
for col in df.columns:
    if 'logradouro' in col.lower():
        col_logradouro = col
        break

if not col_logradouro:
    print("⚠️  Colunas disponíveis no arquivo:")
    for col in df.columns:
        print(f"  - {col}")
    raise Exception("Coluna 'Logradouro' não encontrada! Verifique o nome da coluna.")

print(f"✓ Coluna identificada: '{col_logradouro}'")

# %% [markdown]
# # 4. Agrupar Endereços Únicos

# %%
print("\n" + "=" * 80)
print("PASSO 1: AGRUPAR ENDEREÇOS ÚNICOS")
print("=" * 80)

# Limpar e agrupar
df['logradouro_limpo'] = df[col_logradouro].astype(str).str.strip()
enderecos_unicos = sorted(df['logradouro_limpo'].unique())

print(f"\n✓ Total de registros: {len(df):,}")
print(f"✓ Endereços únicos: {len(enderecos_unicos):,}")
print(f"✓ Redução: {(1 - len(enderecos_unicos)/len(df))*100:.1f}%")

# %% [markdown]
# # 5. Carregar ou Limpar Cache

# %%
print("\n" + "=" * 80)
print("GERENCIAR CACHE")
print("=" * 80)

cache = {}

if cache_file.exists():
    try:
        with open(cache_file, 'r', encoding='utf-8') as f:
            cache = json.load(f)
        print(f"\n✓ Cache encontrado: {len(cache)} endereços")
        
        limpar = input("\nDeseja LIMPAR cache e reprocessar? (s/n): ")
        if limpar.lower() == 's':
            cache = {}
            print("✓ Cache limpo!")
        else:
            print("✓ Cache mantido")
    except Exception as e:
        print(f"⚠️  Erro ao carregar cache: {e}")
        cache = {}
else:
    print("\n✓ Novo cache será criado")

# %% [markdown]
# # 6. Funções de Processamento MELHORADAS

# %%
print("\n" + "=" * 80)
print("FUNÇÕES DE PROCESSAMENTO MELHORADAS")
print("=" * 80)

def extrair_numero_parser(endereco):
    """Extrai número do endereço usando regex"""
    if not isinstance(endereco, str):
        return None
        
    padroes = [
        r'[,:\s](\d+)(?:\s|$|,)',
        r'n[º°]\s*(\d+)',
        r'número\s*(\d+)',
        r'(\d+)[º°]?\s*(?:$|,)',
    ]
    
    for padrao in padroes:
        match = re.search(padrao, endereco, re.IGNORECASE)
        if match:
            return match.group(1)
    return None

def extrair_tipo_logradouro(nome_completo):
    """Extrai tipo do logradouro"""
    if not isinstance(nome_completo, str):
        return None
        
    tipos = {
        'rua': 'Rua', 'avenida': 'Avenida', 'av': 'Avenida', 'av.': 'Avenida',
        'alameda': 'Alameda', 'al': 'Alameda', 'al.': 'Alameda',
        'praça': 'Praça', 'praca': 'Praça', 'pc': 'Praça', 'pc.': 'Praça',
        'largo': 'Largo', 'lg': 'Largo', 'lg.': 'Largo',
        'travessa': 'Travessa', 'trav': 'Travessa', 'tv': 'Travessa', 'tv.': 'Travessa',
        'viaduto': 'Viaduto', 'vd': 'Viaduto', 'vd.': 'Viaduto',
        'passarela': 'Passarela', 'pass': 'Passarela',
        'terminal': 'Terminal', 'term': 'Terminal',
        'estação': 'Estação', 'estacao': 'Estação', 'est': 'Estação', 'est.': 'Estação'
    }
    
    nome_lower = nome_completo.lower().strip()
    for tipo_key, tipo_value in tipos.items():
        if nome_lower.startswith(tipo_key + ' ') or nome_lower == tipo_key:
            return tipo_value
    return None

def normalizar_endereco_sp(endereco):
    """Normaliza endereço para busca em São Paulo"""
    if not isinstance(endereco, str):
        return endereco
    
    # Remover espaços extras
    endereco = re.sub(r'\s+', ' ', endereco.strip())
    
    # Se já tem São Paulo na string, não duplicar
    if 'são paulo' in endereco.lower() or 'sao paulo' in endereco.lower():
        return endereco
    
    return endereco

def buscar_endereco_estrategias_simples(endereco):
    """
    Busca endereço usando estratégias simples mas eficazes para SP
    Retorna (location, strategy_used)
    """
    endereco_normalizado = normalizar_endereco_sp(endereco)
    
    strategies = [
        # Estratégia 1: Endereço completo com contexto SP forte
        {
            'query': f"{endereco_normalizado}, Centro, São Paulo, SP, Brasil",
            'name': 'completo_centro_sp'
        },
        # Estratégia 2: Endereço completo com contexto SP geral
        {
            'query': f"{endereco_normalizado}, São Paulo, SP, Brasil", 
            'name': 'completo_sp'
        },
        # Estratégia 3: Endereço sem número com contexto SP
        {
            'query': f"{re.sub(r'[,\s]\d+.*$', '', endereco_normalizado).strip()}, São Paulo, SP, Brasil",
            'name': 'sem_numero_sp'
        },
        # Estratégia 4: Busca apenas pelo nome + SP (para casos difíceis)
        {
            'query': f"{re.sub(r'[,\s]\d+.*$', '', endereco_normalizado).strip()} São Paulo Brasil",
            'name': 'nome_apenas_sp'
        }
    ]
    
    for strategy in strategies:
        try:
            time.sleep(1.1)  # Respeitar limite do Nominatim
            
            location = geolocator.geocode(
                query=strategy['query'],
                exactly_one=True,
                addressdetails=True,
                country_codes='br'  # Restringir ao Brasil
            )
            
            if location:
                # Verificar se está em São Paulo
                address = location.raw.get('address', {})
                cidade = address.get('city') or address.get('town') or address.get('municipality')
                
                if cidade in ['São Paulo', 'Sao Paulo']:
                    return location, strategy['name']
                elif strategy['name'] == 'nome_apenas_sp':
                    # Última estratégia, aceitar mesmo que não seja SP para análise
                    return location, f"nome_apenas_nao_sp_{cidade}"
                    
        except (GeocoderTimedOut, GeocoderServiceError) as e:
            print(f"   ⚠️ Erro na estratégia {strategy['name']}: {e}")
            continue
        except Exception as e:
            print(f"   ⚠️ Erro inesperado na estratégia {strategy['name']}: {e}")
            continue
    
    return None, "nenhuma"

def geocodificar_endereco(endereco):
    """
    Geocodifica endereço com estratégias simples para São Paulo
    """
    if not isinstance(endereco, str) or not endereco.strip():
        return criar_resultado_nao_encontrado(endereco)
    
    print(f"   🔍 Buscando: {endereco}")
    
    location, estrategia = buscar_endereco_estrategias_simples(endereco)
    
    if not location:
        print(f"   ❌ Não encontrado")
        return criar_resultado_nao_encontrado(endereco)
    
    address = location.raw.get('address', {})
    cidade = address.get('city') or address.get('town') or address.get('municipality')
    
    # VALIDAR SÃO PAULO
    if cidade not in ['São Paulo', 'Sao Paulo']:
        print(f"   🚫 Encontrado em {cidade} (estratégia: {estrategia})")
        return criar_resultado_cidade_incorreta(endereco, cidade, estrategia)
    
    print(f"   ✅ Encontrado em São Paulo (estratégia: {estrategia})")
    
    # EXTRAIR DADOS
    nome_padronizado = (
        address.get('road') or 
        address.get('square') or 
        address.get('pedestrian') or 
        address.get('footway') or
        endereco
    )
    
    numero_api = address.get('house_number')
    numero_parser = extrair_numero_parser(endereco)
    numero_final = numero_api or numero_parser
    
    tipo = extrair_tipo_logradouro(nome_padronizado)
    nome_sem_tipo = nome_padronizado
    if tipo and isinstance(nome_padronizado, str):
        # Remover o tipo do início do nome
        nome_sem_tipo = re.sub(f'^{re.escape(tipo)}\\s*', '', nome_padronizado, flags=re.IGNORECASE).strip()
    
    # Determinar confiança baseado na estratégia e dados encontrados
    if numero_api and 'centro' in estrategia:
        confianca = 'alta'
    elif numero_api:
        confianca = 'media_alta'
    elif numero_parser and 'centro' in estrategia:
        confianca = 'media'
    elif numero_parser:
        confianca = 'media_baixa'
    else:
        confianca = 'baixa'
    
    return {
        'logradouro_original': endereco,
        'tipo_logradouro': tipo,
        'nome_logradouro': nome_sem_tipo,
        'numero_logradouro': numero_final,
        'bairro': address.get('suburb') or address.get('neighbourhood') or address.get('quarter'),
        'cidade': cidade,
        'estado': address.get('state'),
        'latitude': location.latitude,
        'longitude': location.longitude,
        'endereco_completo': location.address,
        'fonte_dados': 'geopy',
        'estrategia': estrategia,
        'confianca': confianca,
        'status': 'sucesso'
    }

def criar_resultado_nao_encontrado(endereco):
    """Resultado para endereços não encontrados"""
    numero = extrair_numero_parser(endereco) if isinstance(endereco, str) else None
    
    return {
        'logradouro_original': endereco,
        'tipo_logradouro': None,
        'nome_logradouro': endereco,
        'numero_logradouro': numero,
        'bairro': None,
        'cidade': None,
        'estado': None,
        'latitude': None,
        'longitude': None,
        'endereco_completo': None,
        'fonte_dados': 'nao_encontrado',
        'estrategia': 'nenhuma',
        'confianca': 'baixa',
        'status': 'nao_encontrado'
    }

def criar_resultado_cidade_incorreta(endereco, cidade_encontrada, estrategia):
    """Resultado para endereços de outras cidades"""
    numero = extrair_numero_parser(endereco) if isinstance(endereco, str) else None
    
    return {
        'logradouro_original': endereco,
        'tipo_logradouro': None,
        'nome_logradouro': endereco,
        'numero_logradouro': numero,
        'bairro': None,
        'cidade': None,
        'estado': None,
        'latitude': None,
        'longitude': None,
        'endereco_completo': None,
        'fonte_dados': 'nao_encontrado',
        'estrategia': estrategia,
        'confianca': 'baixa',
        'status': f'rejeitado_cidade: {cidade_encontrada}'
    }

print("\n✓ Funções de processamento SIMPLIFICADAS carregadas")
print("✓ Estratégias implementadas:")
print("  1. Endereço completo + Centro SP (prioritária)")
print("  2. Endereço completo + SP geral") 
print("  3. Endereço sem número + SP")
print("  4. Nome apenas + SP (fallback)")

# %% [markdown]
# # 7. Geocodificar Endereços

# %%
print("\n" + "=" * 80)
print("PASSO 2-6: GEOCODIFICAR E VALIDAR")
print("=" * 80)

# Filtrar endereços a processar
enderecos_processar = [e for e in enderecos_unicos if e not in cache]

print(f"\n📍 Endereços únicos: {len(enderecos_unicos):,}")
print(f"✓ Já em cache: {len(cache):,}")
print(f"🔄 A processar: {len(enderecos_processar):,}")

if len(enderecos_processar) > 0:
    tempo_estimado = len(enderecos_processar) * 1.1 / 60
    print(f"⏱️  Tempo estimado: ~{tempo_estimado:.1f} minutos")
    
    continuar = input("\nContinuar? (s/n): ")
    
    if continuar.lower() == 's':
        print("\n🔄 Processando...\n")
        
        for i, endereco in enumerate(enderecos_processar, 1):
            print(f"[{i}/{len(enderecos_processar)}] {endereco[:60]}...")
            
            resultado = geocodificar_endereco(endereco)
            cache[endereco] = resultado
            
            # Salvar cache a cada 10
            if i % 10 == 0:
                with open(cache_file, 'w', encoding='utf-8') as f:
                    json.dump(cache, f, ensure_ascii=False, indent=2)
                print(f"   ✓ Cache salvo ({len(cache)} endereços)")
            
            # Progresso a cada 50
            if i % 50 == 0:
                print(f"\n   📊 {i/len(enderecos_processar)*100:.1f}% concluído\n")
        
        # Salvar cache final
        with open(cache_file, 'w', encoding='utf-8') as f:
            json.dump(cache, f, ensure_ascii=False, indent=2)
        
        print(f"\n✓ Geocodificação concluída!")
        print(f"✓ Cache salvo: {len(cache)} endereços")
else:
    print("\n✓ Todos em cache!")

# %% [markdown]
# # 8. Aplicar Dados na Planilha Original

# %%
print("\n" + "=" * 80)
print("ENRIQUECENDO PLANILHA ORIGINAL")
print("=" * 80)

# Criar colunas (ADICIONAMOS 'estrategia')
colunas_novas = [
    'tipo_logradouro', 'nome_logradouro', 'numero_logradouro',
    'bairro', 'cidade', 'estado', 'latitude', 'longitude',
    'endereco_completo', 'fonte_dados', 'estrategia', 'confianca', 'status'
]

for col in colunas_novas:
    df[col] = None

# Aplicar cache
for idx, row in df.iterrows():
    endereco = row['logradouro_limpo']
    
    if endereco in cache:
        dados = cache[endereco]
        for col in colunas_novas:
            df.at[idx, col] = dados.get(col)

print(f"\n✓ Planilha enriquecida com {len(colunas_novas)} novas colunas")
print(f"✓ Nova coluna: 'estrategia' - mostra qual estratégia foi usada na geocodificação")

# %% [markdown]
# # 9. Estatísticas MELHORADAS

# %%
print("\n" + "=" * 80)
print("ESTATÍSTICAS DETALHADAS")
print("=" * 80)

total = len(df)
sucesso = len(df[df['status'] == 'sucesso'])
nao_encontrado = len(df[df['status'] == 'nao_encontrado'])
rejeitado = len(df[df['status'].str.contains('rejeitado', na=False)]) if 'status' in df.columns else 0
erro = len(df[df['status'].str.contains('erro', na=False)]) if 'status' in df.columns else 0

# Estatísticas de estratégias
if 'estrategia' in df.columns:
    estrategias = df['estrategia'].value_counts()
    print(f"\n🎯 ESTRATÉGIAS UTILIZADAS:")
    for estrategia, count in estrategias.items():
        percentual = count/total*100
        print(f"   {estrategia}: {count:,} ({percentual:.1f}%)")

# Confiança
alta = len(df[df['confianca'] == 'alta']) if 'confianca' in df.columns else 0
media_alta = len(df[df['confianca'] == 'media_alta']) if 'confianca' in df.columns else 0
media = len(df[df['confianca'] == 'media']) if 'confianca' in df.columns else 0
media_baixa = len(df[df['confianca'] == 'media_baixa']) if 'confianca' in df.columns else 0
baixa = len(df[df['confianca'] == 'baixa']) if 'confianca' in df.columns else 0

print(f"""
📊 RESULTADOS:

Total: {total:,} registros

STATUS:
✓ Sucesso (São Paulo): {sucesso:,} ({sucesso/total*100:.1f}%)
⚠️  Não encontrado: {nao_encontrado:,} ({nao_encontrado/total*100:.1f}%)
🚫 Rejeitado (outra cidade): {rejeitado:,} ({rejeitado/total*100:.1f}%)
❌ Erro: {erro:,} ({erro/total*100:.1f}%)

CONFIANÇA:
🟢 Alta: {alta:,} ({alta/total*100:.1f}%)
🟡 Média Alta: {media_alta:,} ({media_alta/total*100:.1f}%)
🟡 Média: {media:,} ({media/total*100:.1f}%)
🟠 Média Baixa: {media_baixa:,} ({media_baixa/total*100:.1f}%)
🔴 Baixa: {baixa:,} ({baixa/total*100:.1f}%)
""")

# Exemplos de sucesso por estratégia
print("✅ EXEMPLOS DE SUCESSO POR ESTRATÉGIA:\n")
if 'status' in df.columns and 'estrategia' in df.columns:
    df_sucesso = df[df['status'] == 'sucesso']
    if len(df_sucesso) > 0:
        for estrategia in df_sucesso['estrategia'].unique()[:3]:
            exemplos = df_sucesso[df_sucesso['estrategia'] == estrategia].head(1)
            for _, row in exemplos.iterrows():
                print(f"   Estratégia: {row['estrategia']}")
                print(f"   Original: {row[col_logradouro]}")
                print(f"   Padronizado: {row['tipo_logradouro']} {row['nome_logradouro']}, {row['numero_logradouro'] or 'S/N'}")
                print(f"   Bairro: {row['bairro']} | Confiança: {row['confianca']}\n")
else:
    print("   Nenhum dado de sucesso disponível\n")

# %% [markdown]
# # 10. Exportar Lista de Problemáticos

# %%
print("\n" + "=" * 80)
print("PASSO 7: EXPORTAR ENDEREÇOS PROBLEMÁTICOS")
print("=" * 80)

problematicos_file = pasta_geocoded / 'enderecos_problematicos.txt'

# CORREÇÃO: Validar se as colunas existem antes de acessá-las
with open(problematicos_file, 'w', encoding='utf-8') as f:
    f.write("=" * 100 + "\n")
    f.write("ENDEREÇOS PROBLEMÁTICOS - NECESSITAM REVISÃO\n")
    f.write("=" * 100 + "\n\n")
    
    # 1. Rejeitados
    if 'status' in df.columns:
        df_rejeitados = df[df['status'].str.contains('rejeitado', na=False)]
        if len(df_rejeitados) > 0:
            f.write(f"🚫 REJEITADOS (OUTRA CIDADE): {len(df_rejeitados)} registros\n")
            f.write("=" * 100 + "\n")
            
            # CORREÇÃO: Usar a coluna original em vez de 'logradouro_original'
            unicos = df_rejeitados[[col_logradouro, 'status']].drop_duplicates()
            for _, row in unicos.iterrows():
                cidade = row['status'].replace('rejeitado_cidade: ', '')
                f.write(f"\n{row[col_logradouro]}\n")
                f.write(f"  → Encontrado em: {cidade}\n")
            
            f.write("\n\n")
    
    # 2. Não encontrados
    if 'status' in df.columns:
        df_nao_enc = df[df['status'] == 'nao_encontrado']
        if len(df_nao_enc) > 0:
            f.write(f"⚠️ NÃO ENCONTRADOS: {len(df_nao_enc)} registros\n")
            f.write("=" * 100 + "\n")
            
            # CORREÇÃO: Usar a coluna original
            freq = df_nao_enc[col_logradouro].value_counts().head(50)
            for endereco, count in freq.items():
                f.write(f"\n{endereco} (aparece {count}x)\n")
            
            f.write("\n\n")
    
    # 3. Confiança baixa
    if 'confianca' in df.columns:
        df_baixa = df[df['confianca'] == 'baixa']
        if len(df_baixa) > 0:
            f.write(f"🔴 CONFIANÇA BAIXA: {len(df_baixa)} registros\n")
            f.write("=" * 100 + "\n")
            f.write("(Geocodificados mas sem número identificado)\n\n")
            
            # CORREÇÃO: Usar a coluna original
            unicos_baixa = df_baixa[col_logradouro].drop_duplicates().head(30)
            for endereco in unicos_baixa:
                f.write(f"  - {endereco}\n")

print(f"\n✓ Lista exportada: {problematicos_file}")

# %% [markdown]
# # 11. Exportar Planilha Enriquecida

# %%
print("\n" + "=" * 80)
print("PASSO 8: EXPORTAR PLANILHA ENRIQUECIDA")
print("=" * 80)

output_file = pasta_geocoded / f'Contagem_enriquecida_{datetime.now().strftime("%Y%m%d_%H%M")}.xlsx'

# Remover coluna auxiliar
df_export = df.drop(columns=['logradouro_limpo'])

df_export.to_excel(output_file, index=False)

print(f"\n✓ Planilha exportada: {output_file}")
print(f"✓ Total de colunas: {len(df_export.columns)}")
print(f"✓ Total de linhas: {len(df_export):,}")

# %% [markdown]
# # 12. Resumo Final

# %%
print("\n" + "=" * 80)
print("RESUMO FINAL")
print("=" * 80)

print(f"""
GEOCODIFICAÇÃO E ENRIQUECIMENTO - CONCLUÍDO

ARQUIVOS GERADOS:
✓ {cache_file}
✓ {problematicos_file}
✓ {output_file}

DADOS:
• Registros totais: {len(df):,}
• Endereços únicos: {len(enderecos_unicos):,}
• Taxa de sucesso: {sucesso/total*100:.1f}%

PRÓXIMOS PASSOS:
1. Revisar arquivo de problemáticos
2. Adicionar correções/normalizações se necessário
3. Reprocessar endereços problemáticos
4. Usar planilha enriquecida nos próximos processamentos

COLUNAS ADICIONADAS:
✓ tipo_logradouro
✓ nome_logradouro
✓ numero_logradouro
✓ bairro, cidade, estado
✓ latitude, longitude
✓ endereco_completo
✓ fonte_dados (geopy/nao_encontrado)
✓ estrategia (método de busca usado)
✓ confianca (alta/media_alta/media/media_baixa/baixa)
✓ status
""")

print("=" * 80)
print("✓ PROCESSO CONCLUÍDO!")
print(f"✓ {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}")
print("=" * 80)

GEOCODIFICAÇÃO E ENRIQUECIMENTO DE ENDEREÇOS - v2.2

✓ Bibliotecas importadas
✓ Iniciado em: 22/10/2025 18:36:32

CONFIGURANDO AMBIENTE

✓ Estrutura de pastas:
  - Raiz: c:\Users\x504693\Documents\projetos\projeto_etl_dados
  - Raw: c:\Users\x504693\Documents\projetos\projeto_etl_dados\data\raw
  - Geocoded: c:\Users\x504693\Documents\projetos\projeto_etl_dados\data\geocoded
✓ Geocodificador: Nominatim (OpenStreetMap)
⚠️  Limite: 1 requisição/segundo

CARREGANDO PLANILHA

📁 Arquivos disponíveis em c:\Users\x504693\Documents\projetos\projeto_etl_dados\data\raw:

  1. Contagem diária - Compilado.xlsx
     ↳ Modificado: 22/10/2025 15:51 | Tamanho: 1.3 MB
  2. Base Okuhara Kohei.xlsx
     ↳ Modificado: 21/10/2025 18:54 | Tamanho: 0.0 MB
  3. nomes_Abordados.xlsx
     ↳ Modificado: 21/10/2025 17:51 | Tamanho: 0.0 MB
  4. Base Tablet.xlsx
     ↳ Modificado: 21/10/2025 17:48 | Tamanho: 3.0 MB
  5. CONTAGEM 2025 - CnR.xlsx
     ↳ Modificado: 16/10/2025 17:37 | Tamanho: 0.1 MB


✓ Selecionado: 